<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js115_maskEffectMSC2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js115_maskEffectMSC2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.15 created,  last updated on 2025.01.11
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js115_maskEffectMSC2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js115_maskEffectMSC2D
//    Copyright(C) 2021-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2021.01.08 created, last updated on 2021.02.18
//    ver 0.0.1  2021.10.21 v1, last updated on 2021.10.21
//    ver 0.0.2  2021.11.08 v2, last updated on 2021.11.08
//    ver 0.0.3  2023.05.22 v3, last updated on 2023.09.13
//
// -------------------- SEIR infection model - monte-Cerlo simulation
//
//  S-E-I-R model
//
//      [S] --- beta*I ---> [E] --- sigma ---> [I]--- gamma ---> [R]
//
//
//    S-E-I-R model -- differential equation
//      dS/dt = -beta*S*I
//      dE/dt = beta*S*I - sigma*E
//      dI/dt = sigma*E - gamma*I
//      dR/dt = gamma*I
//    where
//      S: susceptible
//      E: exposed
//      I: infectious
//      R: removed (recover + dead)
//      N = S + E + I + R = constant
//      beta: infection rate (1/day)
//      sigma: incidence rate (1/day) ~ 1/(mean incubation period)
//      gamma: recovery rate (1/day) ~ 1/(mean infected period)
//      mue: birth rate = death rate (1/day) ~ 1/(mean life period)
//
//    basic reproduction number R0
//      R0 = beta*S0/gamma
//     (R0 ~ 2~3 for influenza)  -- infection model
//
//
//    Monte-Cerlo simulation: mask model
//
//      contacted walkers break to 3 cases              infection probability
//
//        1 both walkers with no mask                      p
//          (walker)       (walker)
//
//        2 one walker with a mask and other with no mask  p x maskFactor
//          (walker) ]     (walker) or
//          (walker)     [ (walker)
//
//        3 both walkers with a mask                       p x maskFactor x maskFactor
//          (walker) ]   [ (walker)
//
//      mask ratio
//
//          maskRatio = (walker with a mask) / ( (walker with a mask) + (walker with no mask) )
//
//
// procedure in js code

	let nContact = 0;

	function timeStep() {

		let onset = 0;
		for (let ipp=1; ipp<=nPop; ipp++) {

			// change walker status
			const st = status[ipp]; // status[ipp] 0:susceptible, 1:exposed, 2:infectius, 3:removed
			if (st==1) { // 1:exposed  [E] --- sigma ---> [I]
				if (Math.random()<sigma*dt) {
					status[ipp] = 2; // 2:infectius
					onset++;
				}
			} else if (st==2) {  // 2:infectius [I] --- gamma ---> [R]
				if (Math.random()<gamma*dt) {
					status[ipp] = 3; // 3:removed
				}
			}

			// move walker
			moveWalker(ipp);
		}
		return onset;
	}

	// evolve ipp-walker
	function moveWalker(ipp) {

		const dir = Math.floor(Math.random()*4);

		let ixx, iyy;
		if (dir==0) { // east
			ixx = (ix[ipp]+1)%NNx; iyy = iy[ipp];
		} else if (dir==1) { // west
			ixx = (ix[ipp]-1+NNx)%NNx; iyy = iy[ipp];
		} else if (dir==2) { // north
			ixx = ix[ipp]; iyy = (iy[ipp]+1)%NNy;
		} else if (dir==3) { // south
			ixx = ix[ipp]; iyy = (iy[ipp]-1+NNy)%NNy;
		}

		const f = field[ixx][iyy]; // field[][] 1...NNp: particle, 0:free space, -1:wall
		if (f==0) { // free space
			// ipp-walker move
			field[ix[ipp]][iy[ipp]] = 0;
			ix[ipp] = ixx; iy[ipp] = iyy;
			field[ix[ipp]][iy[ipp]] = ipp;
		} else if (f>0) { // other particle
			// contact
			nContact++;
			// status[ipp] 0:susceptible, 1:exposed, 2:infectius, 3:removed

			// ------------------------------------------------------------------------- js114 no mask effect
			if (status[ipp]==0 && status[f]==2) { // contact S ===> I
				if (Math.random()<0.5*infectiusProb) {
					status[ipp] = 1; // 1:exposed  [S] --- 0.5*infectiusProb ---> [I]
				}
			} else if (status[ipp]==2 && status[f]==0) { // contact I ===> S
				if (Math.random()<0.5*infectiusProb) {
					status[f] = 1; // 1:exposed  [S] --- 0.5*infectiusProb ---> [I]
				}
			}

			// -------------------------------------------------------------------------- js115 with mask effect
			if (status[ipp]==0 && status[f]==2) { // contact S --> I
				p = 0.5*infectiusProb;
				if (maskq[ipp]==1) p = p*maskFactor;
				if (maskq[f]==1) p = p*maskFactor;
				if (Math.random()<p) {
					status[ipp] = 1; // 1:exposed
				}
			}
			if (status[ipp]==2 && status[f]==0) { // contact S --> I
				p = 0.5*infectiusProb;
				if (maskq[ipp]==1) p = p*maskFactor;
				if (maskq[f]==1) p = p*maskFactor;
				if (Math.random()<p) {
					status[f] = 1; // 1:exposed
				}
			}

			// ---------------------------------

		} else if (f==-1) { // wall
			; // ipp-walker not move
		}
	}

//
//    basic reproduction number R0 of this model
//      R0 = numberOfContactToSusceptibleWalkerPerDay*infectiusProb/gamma
//         = ((SS0/(NNx*NNy))/dt)*infectiusProb/gamma
//
//
// --------------------
*/

const maskEffectMSC2D = (function(){ // ====================  maskEffectMSC2D Module  ====================

	const g_nMax = 100000;					// walker array max
	const g_ntMax = 21600;					// max times of data record array (step dt)
	const g_dayvdt = 20;					// 1(day) = g_dayvdt*dt

	let g_NNp = 40000;						// number of walker
	let g_NNx = 360;						// x-division of the field
	let g_NNy = 360;						// y-division of the field

	let g_sysTime = 0.0;					// (day) system time
	let g_dt = 1.0/g_dayvdt;				// (day) time step dt
	let g_incubationPeriod = 5.0;			// (day) mean incubation period
	let g_infectedPeriod = 10.0;			// (day) mean infected period
	let g_infectiusProb = 0.04;				// infectius probability per one contact
	let g_sigma = 1.0/g_incubationPeriod;	// (1/day) incidence rate ~ 1/(mean incubation period) = 1/(5days)
	let g_gamma = 1.0/g_infectedPeriod;		// (1/day) recovery rate ~ 1/(mean infected period) = 1/(10days)

	let g_maskFactor = 0.7;					// (0.0 ... 1.0), infectius probability = maskFactor*(infectiusProb)
	let g_maskRatio = 0.0;					// = (masked person)/(all)

	let g_nPop = g_NNp;						// nPop = S+E+I+R number of walker
	let g_susceptible = g_nPop-30;			// number of susceptible walker
	let g_exposed = 30;						// number of exposed walker
	let g_infectius = 0;					// number of infectius walker
	let g_removed = 0;						// number of removed walker
	let g_onsetPerson = 0;					// number of onset walker
	let g_nContact = 0;						// total number of contact per dt ( nContact ~ NNp*(NNp/(NNx*NNy)) )
	let g_removedRatio = 0.0;
	let g_ss0 = g_susceptible;

	const g_field = dimInt2( g_NNx, g_NNy );	// g_field[i][j]  1...NNp:walker number,  0:free space, -1:wall
	const g_status = dimInt1( g_nMax );			// g_status[i] 0:susceptible, 1:exposed, 2:infectius, 3:removed
	const g_maskq = dimInt1( g_nMax );			// g_maskq = 0:with no mask,  1:with a mask
	const g_ix = dimInt1( g_nMax );				// g_ix[i] x-position of i-th walker
	const g_iy = dimInt1( g_nMax );				// g_iy[i] y-position of i-th walker

	const g_pS = dimInt1( g_ntMax );			// (person) susceptible
	const g_pE = dimInt1( g_ntMax );			// (person) exposed
	const g_pI = dimInt1( g_ntMax );			// (person) infectius
	const g_pR = dimInt1( g_ntMax );			// (person) removed
	const g_pN = dimInt1( g_ntMax );			// (person) total = S+E+I+R
	const g_onsetvdt = dimInt1( g_ntMax );		// (person) onset person per dt
	const g_onsetvday = dimInt1( g_ntMax );		// (person) onset person per day
	const g_nContactvdt = dimInt1( g_ntMax );	// number of contact per dt

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [] ;
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio ) {

		setParam( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio );
		g_removedRatio = 0.0;

		g_sysTime = 0;						// (day) system time

		g_nPop = g_NNp;						// nPop = S + E + I + R
		g_removed = Math.floor(g_nPop*g_removedRatio);	// number of removed walkers
		g_exposed = 30;						// number of exposed walkers
		g_susceptible = g_nPop - g_removed - g_exposed;	// number of susceptible walkers
		g_infectius = 0;					// number of infectius walkers
		g_onsetPerson = 0;					// number of onset walker
		g_nContact = 0;						// number of contact per dt
		g_ss0 = g_susceptible;				// S(0): initial number of susceptible walkers

		clearField();
		clearPop();
		setWalkers(g_nPop,g_exposed);
		setMask(g_nPop,g_maskRatio);
	}

	function setParam( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio ) {
		g_infectiusProb = v_infectiusProb;// infectius probability per one contact
		g_sigma = v_sigma;				// (1/day) incidence rate ~ 1/(mean incubation period) = 1/(5days)
		g_gamma = v_gamma;				// (1/day) recovery rate ~ 1/(mean infected period) = 1/(10days)
		g_NNp = v_popDens;				// nPop = S+E+I+R number of walkers
		g_maskFactor = v_maskFactor;	// (0.0 ... 1.0), infectius probability = maskFactor*(infectiusProb)
		g_maskRatio = v_maskRatio;		// = (masked person)/(all)

		setMask(g_nPop,g_maskRatio);
	}

	function clearField() {
		const nnx=g_NNx, nny=g_NNy;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_field[i][j] = 0;
			}
		}
	}

	function clearPop() {
		const nt=g_ntMax;
		for (let i=0; i<nt; i++) {
			g_pS[i] = 0;
			g_pE[i] = 0;
			g_pI[i] = 0;
			g_pR[i] = 0;
			g_pN[i] = 0;
			g_onsetvdt[i] = 0;
			g_onsetvday[i] = 0;
			g_nContactvdt[i] = 0;
		}
	}

	function setWalkers(nn, initExposedNum) {
		const nnx=g_NNx, nny=g_NNy;

		const rr0 = nn*g_maskRatio;
		// set susceptible nn walkers
		for (let i=1; i<=nn; i++) {
			let x, y;
			do {
				x = Math.floor(nnx*Math.random());
				y = Math.floor(nny*Math.random());
			} while (g_field[x][y]!=0); // g_field[][]: free space: 0, walker:1...NNp-1
			g_field[x][y] = i;
			g_ix[i] = x;
			g_iy[i] = y;
			g_status[i] = 0; // 0:susceptible

			if (i<=rr0) {
				g_maskq[i] = 1; // with a mask
			} else {
				g_maskq[i] = 0; // with no mask
			}
		}

		// set exposed initExposedNum walkers
		for (let i=0; i<initExposedNum; i++) {
			let ipp;
			do {
				ipp = Math.floor(nn*Math.random()) + 1;
			} while (g_status[ipp]!=0); //g_status[ipp] 0:susceptible, 1:exposed, 2:infectius, 3:removed
			g_status[ipp] = 1; // 1:exposed
		}
	}

	function setMask(nn,maskRatio) {
		const nMask = nn*maskRatio;
		for (let i=1; i<=nn; i++) {
			if (i<=nMask) {
				g_maskq[i] = 1; // with a mask
			} else {
				g_maskq[i] = 0; // with no mask
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc ) {
		for (let i=0; i<nCalc; i++) {
			const it = Math.floor(g_sysTime/g_dt);
			memoSEIRN(it); // set g_pS[it],g_pE[it],g_pI[it],g_pR[it],g_pN[it]
			g_sysTime += g_dt;
			g_onsetPerson = timeStep();
		}
	}

	function timeStep() {
		const nnp=g_NNp;
		let onset=0;
		g_nContact = 0;
		for (let ipp=1; ipp<=nnp; ipp++) {
			const st = g_status[ipp]; // g_status[ipp] 0:susceptible, 1:exposed, 2:infectius, 3:removed
			if (st==1) { // 1:exposed  [E] --- sigma ---> [I]
				if (Math.random()<g_sigma*g_dt) {
					g_status[ipp] = 2; // 2:infectius
					onset++;
				}
			} else if (st==2) {  // 2:infectius [I] --- gamma ---> [R]
				if (Math.random()<g_gamma*g_dt) {
					g_status[ipp] = 3; // 3:removed
				}
			}
			moveWalker(ipp);
		}
		return onset;
	}

	function moveWalker(ipp) {
		const dir = Math.floor(Math.random()*4);
		let ixx, iyy;
		if (dir==0) { // east
			ixx = (g_ix[ipp]+1)%g_NNx; iyy = g_iy[ipp];
		} else if (dir==1) { // west
			ixx = (g_ix[ipp]-1+g_NNx)%g_NNx; iyy = g_iy[ipp];
		} else if (dir==2) { // north
			ixx = g_ix[ipp]; iyy = (g_iy[ipp]+1)%g_NNy;
		} else if (dir==3) { // south
			ixx = g_ix[ipp]; iyy = (g_iy[ipp]-1+g_NNy)%g_NNy;
		}
		const f = g_field[ixx][iyy]; // g_field[][] 1...NNp: particle, 0:free space, -1:wall
		if (f==0) { // free space
			// ipp-walker move
			g_field[g_ix[ipp]][g_iy[ipp]] = 0;
			g_ix[ipp] = ixx; g_iy[ipp] = iyy;
			g_field[g_ix[ipp]][g_iy[ipp]] = ipp;
		} else if (f>0) { // other particle
			// contact
			g_nContact++;
			// g_status[ipp] 0:susceptible, 1:exposed, 2:infectius, 3:removed
			if (g_status[ipp]==0 && g_status[f]==2) { // contact S --> I
				let p = 0.5*g_infectiusProb;
				if (g_maskq[ipp]==1) p = p*g_maskFactor;
				if (g_maskq[f]==1) p = p*g_maskFactor;
				if (Math.random()<p) {
					g_status[ipp] = 1; // 1:exposed
				}
			}
			if (g_status[ipp]==2 && g_status[f]==0) { // contact S --> I
				let p = 0.5*g_infectiusProb;
				if (g_maskq[ipp]==1) p = p*g_maskFactor;
				if (g_maskq[f]==1) p = p*g_maskFactor;
				if (Math.random()<p) {
					g_status[f] = 1; // 1:exposed
				}
			}
		} else if (f==-1) { // wall
			; // ipp-walker not move
		}
	}

	function memoSEIRN(i) {
		const nnp=g_NNp;
		let ss = 0, ee = 0, ii = 0, rr = 0, nn = 0;
		for (let ipp=1; ipp<=nnp; ipp++) {
			const st = g_status[ipp];
			if (st==0) {
				ss++;
			} else if (st==1) {
				ee++;
			} else if (st==2) {
				ii++;
			} else if (st==3) {
				rr++;
			}
		}
		g_susceptible = ss;
		g_exposed = ee;
		g_infectius = ii;
		g_removed = rr;
		g_nPop = ss + ee + ii + rr;

		if (i>g_ntMax-1) return;

		g_pS[i] = g_susceptible;
		g_pE[i] = g_exposed;
		g_pI[i] = g_infectius;
		g_pR[i] = g_removed;
		g_pN[i] = g_nPop;
		g_onsetvdt[i] = g_onsetPerson;
		setPoday(i);
		g_nContactvdt[i] = g_nContact;
	}

	function setPoday(it) {
		let s = 0;
		for (let i=-19; i<=0; i++) {
			const ii = it + i;
			if (ii>=0) {
				s += g_onsetvdt[ii];
			}
		}
		g_onsetvday[it] = s;
	}

	function iNow() { return Math.floor(g_sysTime/g_dt); }


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,
						// init( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio )
		setParam:		setParam,
						// setParam( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio)
		evolve:			timeEvolution,		// timeEvolution( v_nCalc )

		getSysParam:	function() { return [ g_NNx, g_NNy, g_dt, g_NNp, g_ss0 ]; },
		getNow:			function() { return [ g_sysTime, iNow(), g_nContact, g_susceptible ]; },
		getNowSEIRN:	function() { return [ g_susceptible, g_exposed, g_infectius, g_removed, g_nPop ]; },
		getSEIRN:		function(i) { return [ g_pS[i], g_pE[i], g_pI[i], g_pR[i], g_pN[i], g_onsetvday[i] ]; },
		getS:			function(i) { return g_pS[i]; },
		getE:			function(i) { return g_pE[i]; },
		getI:			function(i) { return g_pI[i]; },
		getR:			function(i) { return g_pR[i]; },
		getN:			function(i) { return g_pN[i]; },
		getNewCase:		function(i) { return g_onsetvday[i]; },
		getWalker:		function(i) { return [ g_status[i], g_ix[i], g_iy[i] ]; },
	};

})(); // ====================  maskEffectMSC2D end  ====================


const js115 = (function(){ // ====================  js Module  ====================

	const theModule = maskEffectMSC2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_infectiusProb = 0.04;	// infectius probability per one contact ;
	let v_sigma = 0.2;			// (1/day) incidence rate ~ 1/(mean incubation period) = 1/(5days)
	let v_gamma = 0.1;			// (1/day) recovery rate ~ 1/(mean infected period) = 1/(10days)
	let v_popDens = 40000;		// = S+E+I+R number of walkers
	let v_maskFactor = 0.7;		// (0.0 ... 1.0), infectius probability = maskFactor*(infectiusProb)
	let v_maskRatio = 0.0;		// = (masked person)/(all)
	let v_nCalc = 6;

	let p_NNx, p_NNy, p_dt, p_NNp, p_S0; // <-- theModule.getSysParam()

  let sysTime, iNow, nContact, pS, pE, pI, pR, pN, R0;
  let ssList = [];
  let eeList = [];
  let iiList = [];
  let rrList = [];
  let nnList = [];
  let newCaseList = [];

	let dispMode = 0;
	let timeRange = 360; // (day)

	let paramChanged = false;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;
	let outputFlag = false;

  let breakFlag = false;
  let dataFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g2d.init( canvas, xCanvasSize, yCanvasSize );
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			outputFlag = false;
			theModule.init( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio );
			[ p_NNx, p_NNy, p_dt, p_NNp, p_S0 ] = theModule.getSysParam();
      dataFlag = true;
		}

		if ( paramChanged ) {
			paramChanged = false;
			theModule.setParam( v_infectiusProb,v_sigma,v_gamma,v_popDens,v_maskFactor,v_maskRatio );
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
			//inStepFlag = true;
		}

		draw( ctx, dispMode, timeRange, outputFlag );

    if ( dataFlag ) setPopData();

		requestAnimationFrame(animate);
	}

  function setPopData() {
    ssList = [];
    eeList = [];
    iiList = [];
    rrList = [];
    nnList = [];
    newCaseList = [];
    for (let i=0; i<timeRange; i++) {
      const j = i*20;
      let pS, pE, pI, pR, pN, newCase;
      [ pS, pE, pI, pR, pN, newCase ] = theModule.getSEIRN(j);
      ssList[i] = pS;
      eeList[i] = pE;
      iiList[i] = pI;
      rrList[i] = pR;
      nnList[i] = pN;
      newCaseList[i] = newCase;
    }
	}


	//--------------------  draw  --------------------

	const gColor = {
		S:"#0088ff", Ex:"#ffff00", I:"#ff0000", R:"#00ff00", N:"#ff00ff", NewCase:"#ff8800",
		text:"#888888", grid:"#666666"
	};

	function draw( ctx, dispMode, timeRange, outputFlag ) {
		const xBoxSize = 400, yBoxSize = 400, xp=60, yp=60;
    let susceptible;
		[ sysTime, iNow, nContact, susceptible ] = theModule.getNow();
    [ pS, pE, pI, pR, pN ] = theModule.getNowSEIRN();
    R0 = p_S0/(p_NNx*p_NNy)/p_dt*v_infectiusProb/v_gamma; //R0= ((SS0/(NNx*NNy))/dt)*infectiusProb/gamma

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) {
			plotSEIR(ctx, xp,yp,timeRange);
		} else if (dispMode==1) {
			logPlotSEIR(ctx, xp,yp,timeRange);
		} else if (dispMode==2) {
			drawField( ctx );
		}

		ctx.fillStyle = "#888888";
		ctx.fillText(`time =${sysTime.toFixed(2)},  nContact =${nContact},  R0 =${R0.toFixed(2)}`, 10, yCanvasSize-25);
		ctx.fillText(`field = ${p_NNx} x ${p_NNy},  walker =${v_popDens} `, 10, yCanvasSize-5);

		//if ( outputFlag ) outputTextArea(1,timeRange);
	}


	function drawField( ctx ) {
		const cx0 = xCanvasSize/2+g2d.xShift, cy0 = yCanvasSize/2-30+g2d.yShift, xBoxSize = 360, yBoxSize = 360;
		const scale = 1.0*g2d.zoom;
		const xSize = xBoxSize*scale, ySize = yBoxSize*scale;
		const xp = cx0 - xSize/2, yp = cy0-ySize/2;

		const nnx=p_NNx, nny=p_NNy, nn=p_NNp, ypMax = yp+p_NNy;
		const wColor = [ gColor.S, gColor.Ex, gColor.I, gColor.R  ];

		// box
		ctx.strokeStyle = "#888888";
		ctx.strokeRect( xp-1*scale, yp-1*scale, (nnx+2)*scale, (nny+2)*scale );

		for (let i=0; i<nn; i++) {
			let st, ix, iy; // st:  0:susceptible, 1:exposed, 2:infectius, 3:removed
			[ st, ix, iy ] = theModule.getWalker(i);
			ctx.fillStyle = wColor[st];
			ctx.fillRect( xp+ix*scale, yp+(nny-iy-1)*scale, 1.0*scale, 1.0*scale );
		}

		let pS, pE, pI, pR, pN;
		[ pS, pE, pI, pR, pN ] = theModule.getNowSEIRN();
		ctx.fillStyle = gColor.S;  ctx.fillText( `S: ${pS}`,  20, 25 );
		ctx.fillStyle = gColor.Ex; ctx.fillText( `E: ${pE}`, 100, 25 );
		ctx.fillStyle = gColor.I;  ctx.fillText( `I: ${pI}`, 180, 25 );
		ctx.fillStyle = gColor.R;  ctx.fillText( `R: ${pR}`, 260, 25 );
		ctx.fillStyle = gColor.N;  ctx.fillText( `total: ${pN}`, 340, 25 );
	}


	function plotSEIR(ctx, xp,yp,timeRange) {
		const mag = 300.0/p_NNp, m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,30,"0",timeRange+"(day)","0",`${p_NNp}`, gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Exposed", xp+70, yp-25, gColor.Ex );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "new cases/day x 50", xp+260, yp-25, gColor.NewCase );
		drawText(ctx, "plot SEIR", xp+100, yp+360, gColor.text );

		plotFunc( ctx, xp, yp, m, theModule.getS, mag, gColor.S );
		plotFunc( ctx, xp, yp, m, theModule.getE, mag, gColor.Ex );
		plotFunc( ctx, xp, yp, m, theModule.getI, mag, gColor.I );
		plotFunc( ctx, xp, yp, m, theModule.getR, mag, gColor.R );
		plotFunc( ctx, xp, yp, m, theModule.getNewCase, mag*50, gColor.NewCase );
		plotFunc( ctx, xp, yp, m, theModule.getN, mag, gColor.N );
	}

	function plotFunc(ctx, xp,yp,m,pFunc,mag,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			ctx.lineTo(xp+i, yp+300-mag*pFunc(i*m));
		}
		ctx.stroke();
	}

	function logPlotSEIR(ctx, xp,yp,timeRange) {
		const m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,60,"0",timeRange+"(day)","10^0","10^5", gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Exposed", xp+70, yp-25, gColor.Ex );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "new cases/day", xp+260, yp-25, gColor.NewCase );
		drawText(ctx, "log plot SEIR", xp+100, yp+360, gColor.text );

		plotLog10Func(ctx, xp,yp,m,theModule.getS, gColor.S );
		plotLog10Func(ctx, xp,yp,m,theModule.getE, gColor.Ex );
		plotLog10Func(ctx, xp,yp,m,theModule.getI, gColor.I );
		plotLog10Func(ctx, xp,yp,m,theModule.getR, gColor.R );
		plotLog10Func(ctx, xp,yp,m,theModule.getNewCase, gColor.NewCase );
		plotLog10Func(ctx, xp,yp,m,theModule.getN, gColor.N );
	}

	function plotLog10Func(ctx, xp,yp,m,pFunc,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			let p = pFunc(i*m);
			if (p<=0.0) p = 0.1;
			const logp = Math.log10(p);
			if (logp>-0.2) ctx.lineTo(xp+i, yp+300-60.0*logp);
		}
		ctx.stroke();
	}

	function drawGrid(ctx, xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color) {
		ctx.strokeStyle = color;
		ctx.fillStyle = color;
		ctx.strokeRect(xp,yp,xLen,yLen);

		for (let i=0; i<xLen; i+=xDiv) {
			drawLine(ctx, xp+i,yp, xp+i, yp+yLen,color);
		}
		for (let j=0; j<yLen; j+=yDiv) {
			drawLine(ctx, xp,yp+yLen-j, xp+xLen, yp+yLen-j,color);
		}
		ctx.fillText(x0Str, xp-5, yp+yLen+20);
		ctx.fillText(xMaxStr, xp+xLen-xMaxStr.length*4, yp+yLen+20);
		ctx.fillText(y0Str, xp-y0Str.length*8-12, yp+yLen+5);
		ctx.fillText(yMaxStr, xp-yMaxStr.length*8-12, yp+5);
	}

	 function drawLine( ctx, x1, y1, x2, y2, color ) {
	 	ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}

	function drawText( ctx, txt, x, y, color ) {
		ctx.fillStyle = color;
		ctx.fillText(txt, x, y);
	}


	// --- mouse event  --- mouse drug to shift and mouse wheel to zoom in/out
	// ver 0.0.0  2023.05.16  last updated  2023.05.17
	//
	// extenal xCanvasSize, yCanvasSize

	const g2d = {};
	g2d.mouseDownFlag = 0;			// 1:on mouse down, 0:else
	g2d.x_mouse = 0;				// x-position of mouse
	g2d.y_mouse = 0;				// y-position of mouse
	g2d.x0_mouse = 0;				// drag-started x-position of mouse
	g2d.y0_mouse = 0;				// drag-started y-position of mouse
	g2d.xLimit = xCanvasSize*1.5;	// -xLimit <= xShift <= xLimit
	g2d.yLimit = yCanvasSize*1.5;	// -yLimit <= yShift <= yLimit

	// public
	g2d.zoom = 1.0;
	g2d.xShift = 0.0;
	g2d.yShift = 0.0;

	g2d.init = function( canvas, xSize, ySize ) {
		g2d.setMouseOnCanvas( canvas );
		g2d.xLimit = xSize*1.5;
		g2d.yLimit = ySize*1.5;
	};

	g2d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g2d.mouse_move);
		canvas.addEventListener('mousedown', g2d.mouse_down);
		canvas.addEventListener('mouseup', g2d.mouse_up);
		//canvas.addEventListener("mousewheel", g2d.mouseWheel);
	};

	g2d.mouse_move = function(e) {
		var pi = Math.PI;

		if (g2d.mouseDownFlag==1) {
			g2d.x_mouse = e.clientX;
			g2d.y_mouse = e.clientY;
			g2d.xShift = g2d.xShift + (g2d.x_mouse-g2d.x0_mouse);
			if (g2d.xShift<-g2d.xLimit) g2d.xShift=-g2d.xLimit;
			if (g2d.xShift>g2d.xLimit) g2d.xShift=g2d.xLimit;
			g2d.yShift = g2d.yShift + (g2d.y_mouse-g2d.y0_mouse);
			if (g2d.yShift<-g2d.yLimit) g2d.yShift=-g2d.yLimit;
			if (g2d.yShift>g2d.yLimit) g2d.yShift=g2d.yLimit;
			g2d.x0_mouse = g2d.x_mouse;
			g2d.y0_mouse = g2d.y_mouse;
		}
	};

	g2d.mouse_down = function(e) {
		if (g2d.mouseDownFlag==0) {
			g2d.x0_mouse = e.clientX;
			g2d.y0_mouse = e.clientY;
			g2d.x_mouse = g2d.x0_mouse;
			g2d.y_mouse = g2d.y0_mouse;
			g2d.mouseDownFlag = 1;
		}
	};

	g2d.mouse_up = function(e) {
		if (g2d.mouseDownFlag==1) {
			g2d.mouseDownFlag = 0;
		}
	};

	g2d.mouseWheel = function(e) {
		g2d.deltaY = e.deltaY;
		if ( g2d.deltaY > 0 ) g2d.zoom *= 0.95;
		else if ( g2d.deltaY < 0 ) g2d.zoom *= 1.05;
		if ( g2d.zoom<0.25 ) g2d.zoom = 0.25;
		if ( g2d.zoom>4.0 ) g2d.zoom = 4.0;
	};

	// --- mouse event end


	// --------------------  data output  --------------------

	function outputTextArea(outputSW,timeRange) {
		// outputSW:  0:textarea clear,  1:output text data
		outputFlag = false;

		let str;

		if (outputSW==0) {
			str = "";

		} else if (outputSW==1) {
			const R0 = p_S0/(p_NNx*p_NNy)/p_dt*v_infectiusProb/v_gamma; //R0= ((SS0/(NNx*NNy))/dt)*infectiusProb/gamma
			let pS, pE, pI, pR, pN, newCase; [ pS, pE, pI, pR, pN, newCase ] = theModule.getSEIRN(0);
			str = `infection prob. = ${v_infectiusProb.toFixed(5)},  sigma = ${v_sigma.toFixed(2)} \n`
				+ `gamma = ${v_gamma.toFixed(2)},  R0 = ${R0.toFixed(2)} \n`
				+ `S(0) = ${pS.toFixed(1)},  E(0) = ${pE.toFixed(1)},  I(0) = ${pI.toFixed(1)},`
				+ `  R(0) = ${pR.toFixed(1)} \n \n`;

			str += ` day,  S(t),  E(t),  I(t),  R(t), S+E+I+R, newCase \n`;
			for (let i=0; i<timeRange; i++) {
				const j = i*20;
				[ pS, pE, pI, pR, pN, newCase ] = theModule.getSEIRN(j);
				str += ` ${i},  ${pS.toFixed(1)},  ${pE.toFixed(1)},  ${pI.toFixed(1)},  ${pR.toFixed(1)},`
					+ `  ${pN.toFixed(1)},  ${newCase.toFixed(1)} \n`;
			}
		}
		document.getElementById("outputText").innerHTML = str;
	}


	// --------------------  dom control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
		document.getElementById("home_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setpN0() {
		const n = 1 + document.getElementById("slct_pN0").selectedIndex;
		v_popDens = n*10000;
		resetFlag = true;
	}

	function setInfectProb() {
		const p = 0 + document.getElementById("range_infectiusProb").value;
		v_infectiusProb = 0.001*p;
		document.getElementById("text_infectiusProb").innerHTML = " " + v_infectiusProb.toFixed(3);
		paramChanged = true;
	}

	function setSigma() {
		const s = 0 + document.getElementById("range_sigma").value;
		v_sigma = 0.005*s;
		document.getElementById("text_sigma").innerHTML =
			" " + v_sigma.toFixed(2) + " (1/ " + (1.0/v_sigma).toFixed(1) + " day)";
		paramChanged = true;
	}

	function setGamma() {
		const g = 0 + document.getElementById("range_gamma").value;
		v_gamma = 0.005*g;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		paramChanged = true;
	}

	function setMaskFactor() {
		const r = 0 + document.getElementById("range_maskFactor").value;
		v_maskFactor = 0.01*r;
		document.getElementById("text_maskFactor").innerHTML = " " + v_maskFactor.toFixed(2);
		paramChanged = true;
	}

	function setMaskRatio() {
		const r = 0 + document.getElementById("range_maskRatio").value;
		v_maskRatio = 0.01*r;
		document.getElementById("text_maskRatio").innerHTML = " " + v_maskRatio.toFixed(2);
		paramChanged = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;

		if ( dispMode==2 ) {
			document.getElementById("home_button").style.visibility = "visible";
			document.getElementById('home_caption').innerHTML =
				"drag mouse to shift the box, or wheel to zoom in / out";
		} else {
			document.getElementById("home_button").style.visibility = "hidden";
			document.getElementById('home_caption').innerHTML ="";
		}
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

	function setTimeRange() {
		timeRange = setTimeRange.range[ 0 + document.getElementById("slct_timeRange").selectedIndex ];
	}
	setTimeRange.range = [180,360,720,1080];

	function viewHome() {
		g2d.zoom = 1.0;
		g2d.xShift = 0.0;
		g2d.yShift = 0.0;

	}

	function getData() {
		outputFlag = true;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetBeta( beta ) {
		v_beta = beta;
		document.getElementById("text_beta").innerHTML = " " + v_beta.toFixed(5);
		resetFlag = true;
  }

  function pysetSigma( sigma ) {
		v_sigma = sigma;
		document.getElementById("text_sigma").innerHTML =
			" " + v_sigma.toFixed(2) + " (1/ " + (1.0/v_sigma).toFixed(1) + " day)";
		resetFlag = true;
  }

  function pysetGamma( gamma ) {
		v_gamma = gamma;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		resetFlag = true;
  }

	function pysetMaskFactor( factor ) {
		v_maskFactor = factor;
		document.getElementById("text_maskFactor").innerHTML = " " + v_maskFactor.toFixed(2);
		paramChanged = true;
	}

	function pysetMaskRatio( ratio ) {
		v_maskRatio = ratio;
		document.getElementById("text_maskRatio").innerHTML = " " + v_maskRatio.toFixed(2);
		paramChanged = true;
	}

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetNow() {
    return [ sysTime, iNow, nContact, pS, pE, pI, pR, pN ];
  }

  function pygetData() {
    return [ v_infectiusProb, v_sigma, v_gamma, v_popDens, v_maskFactor, v_maskRatio, R0 ];
  }

  function pygetPopData() {
    dataFlag = false;
    return [ ssList, eeList, iiList, rrList, nnList, newCaseList ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		// dom control
		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setpN0:				setpN0,				// setpN0()
		setInfectProb:		setInfectProb,		// setInfectProb()
		setSigma:			setSigma,			// setSigma()
		setGamma:			setGamma,			// setGamma()
		setMaskFactor:		setMaskFactor,		// setMaskFactor()
		setMaskRatio:		setMaskRatio,		// setMaskRatio()

		setDispMode:		setDispMode,		// setDispMode()
		setSpeed:			setSpeed,			// setSpeed()
		setTimeRange:		setTimeRange,		// setTimeRange()
		viewHome:			viewHome,			// viewHome()
		getData:			getData,			// getData()

    breakLoop: breakLoop, // breakLoop();
    pysetBeta: pysetBeta, // pysetBeta( beta )
    pysetSigma: pysetSigma, // pysetSigma( sigma )
    pysetGamma: pysetGamma, // pysetGamma( gamma )
    pysetMaskFactor: pysetMaskFactor, //pysetMaskFactor( factor )
    pysetMaskRatio: pysetMaskRatio, //pysetMaskRatio( ratio )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetNow: pygetNow, // pygetNow() : return [ sysTime, iNow, nContact, pS, pE, pI, pR, pN ]
    pygetData: pygetData, // pygetData() : return [ infectiusProb, sigma, gamma, popDens, maskFactor, maskRatio, R0 ]
    pygetPopData: pygetPopData, //() :return [ ssList, eeList, iiList, rrList, nnList, newCaseList ]
	};

})(); // ====================  js115 module end  ====================


const js = js115;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js115] mask effect - SEIR model - Monte-Carlo simulation 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<span>theme: mask effect</span>
    <span style="margin-right: 20px;"></span>
<label>walkers:</label>
<select id="slct_pN0" onChange="js.setpN0()">
<option selected>10000</option><option>20000</option><option>30000</option><option selected>40000</option>
<option>50000</option>
</select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>infection probability =</label>
<input type="range" id="range_infectiusProb" min="1" max="100" value="40" step="1"
style="width:200px" oninput="js.setInfectProb()">
<label id="text_infectiusProb"> 0.040</label>(1/contact)
<br>

<label>sigma (incidence rate)=</label>
<input type="range" id="range_sigma" min="1" max="100" value="40" step="1"
style="width:200px" oninput="js.setSigma()">
<label id="text_sigma"> 0.20 (1/ 5.0 day)</label>
<br>

<label>gamma (recovery rate)=</label>
<input type="range" id="range_gamma" min="1" max="100" value="20" step="1"
style="width:200px" oninput="js.setGamma()">
<label id="text_gamma"> 0.10 (1/ 10.0 day)</label>
<br>

<label>mask factor =</label>
<input type="range" id="range_maskFactor" min="0" max="100" value="70" step="1"
style="width:200px" oninput="js.setMaskFactor()">
<label id="text_maskFactor"> 0.70</label>
<br>

<label>mask ratio =</label>
<input type="range" id="range_maskRatio" min="0" max="100" value="0" step="1"
style="width:200px" oninput="js.setMaskRatio()">
<label id="text_maskRatio"> 0.00</label>
<br>

<label>dispMode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>plot SEIR</option>
<option >log plot SEIR</option>
<option >walkers</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>speed:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option selected>6</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>time range:</label>
<select id="slct_timeRange" onChange="js.setTimeRange()">
<option >0 - 180</option><option selected>0 - 360</option><option >0 - 720</option>
<option >0 - 1080</option></select>
(days)
<br>

<button id="home_button" onClick="js.viewHome()">return to initial view</button>
 <span id="home_caption" ></span>
<br>

<!--
<button id="getData_button" onClick="js.getData()">output text data</button>
<br>

<textarea id="outputText" rows="6" cols="70" disabled></textarea>
<br>
-->

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def

mask_ratio = 0

In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data <-- MCS simulator

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get system param and print
print("-- get system parameter and print --")
[ infectiusProb, sigma, gamma, popDens, maskFactor, maskRatio, R0 ] = eval_js('js.pygetData()')
print(f' infectiusProb = {infectiusProb:>5.3f}, sigma = {sigma:>6.4f}, gamma = {gamma:>6.4f}, popDens= {popDens:>6d}, R0 = {R0:5.2f}')
print(f'  maskFactor = {maskFactor:>6.2f}, maskRatio = {maskRatio:>6.2f} \n')

# get S E I R data and print
print("-- get S E I R data and print --")
for i in range(10):
  [ sysTime, iNow, nContact, pS, pE, pI, pR, pN ] = eval_js('js.pygetNow()')
  print(f'time = {sysTime:>6.2f}, iNow = {iNow:>4d}, nContact = {nContact:>6d}, S = {pS:>8.1f}, E = {pE:>8.1f}, I = {pI:>8.1f}, R = {pR:>8.1f}')
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [3]:
# input mask_factor ( 0... 1)

mask_ratio = 0.31 # @param {type:"slider", min:0, max:1, step:0.01}

In [ ]:
# get data <-- MCS simulator

import time

# exec html-js code
exec_html_js()
print("-- start --")

# set mask factor
print("-- set mask ratio :", mask_ratio, " --")
eval_js('js.pysetMaskRatio({})'.format(mask_ratio))

# get system param and print
print("-- get system parameter and print --")
[ infectiusProb, sigma, gamma, popDens, maskFactor, maskRatio, R0 ] = eval_js('js.pygetData()')
print(f' infectiusProb = {infectiusProb:>5.3f}, sigma = {sigma:>6.4f}, gamma = {gamma:>6.4f}, popDens= {popDens:>6d}, R0 = {R0:5.2f}')
print(f'  maskFactor = {maskFactor:>6.2f}, maskRatio = {maskRatio:>6.2f} \n')

# get S E I R data and print
print("-- get S E I R data and print --")
for i in range(10):
  [ sysTime, iNow, nContact, pS, pE, pI, pR, pN ] = eval_js('js.pygetNow()')
  print(f'time = {sysTime:>6.2f}, iNow = {iNow:>4d}, nContact = {nContact:>6d}, S = {pS:>8.1f}, E = {pE:>8.1f}, I = {pI:>8.1f}, R = {pR:>8.1f}')
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get S-E-I-R population <-- MCS simulator

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# set mask factor
mask_ratio = 0.4
print("-- set mask ratio :", mask_ratio, " --")
eval_js('js.pysetMaskRatio({})'.format(mask_ratio))

# get system param and print
print("-- get system parameter and print --")
[ infectiusProb, sigma, gamma, popDens, maskFactor, maskRatio, R0 ] = eval_js('js.pygetData()')
print(f' infectiusProb = {infectiusProb:>5.3f}, sigma = {sigma:>6.4f}, gamma = {gamma:>6.4f}, popDens= {popDens:>6d}, R0 = {R0:5.2f}')
print(f'  maskFactor = {maskFactor:>6.2f}, maskRatio = {maskRatio:>6.2f} \n')

# get S E I R data and print
print("-- get S E I R data and print --")
for i in range(10):
  [ sysTime, iNow, nContact, pS, pE, pI, pR, pN ] = eval_js('js.pygetNow()')
  print(f'time = {sysTime:>6.2f}, iNow = {iNow:>4d}, nContact = {nContact:>6d}, S = {pS:>8.1f}, E = {pE:>8.1f}, I = {pI:>8.1f}, R = {pR:>8.1f}')
  time.sleep(2)

# get S E I R newCase data
print("-- get SEIR and newCase data --")
[ ssList, eeList, iiList, rrList, nnList, newCaseList ] = eval_js('js.pygetPopData()')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

np_data = np.array([ ssList, eeList, iiList, rrList, nnList, newCaseList ])
print(f' np_data shape ; {np_data.shape}')
print("-- end --")

In [ ]:
# save data

import numpy as np

print("-- save data : np.array([ ssList, eeList, iiList, rrList, nnList, newCaseList ]) --")

np.save('js115_data.npy', np_data)

print("-- saved data shape :", np_data.shape, " --")

# The saved file can be viewed by clicking on the folder icon on the left edge of the notebook.
# You must download this file to save it permanently.
# Select Download from the '...'.T menu of the file to download it.

In [7]:
# save data as csv file

import numpy as np

np.savetxt('js115_data.csv', np_data.T, delimiter=',')

In [ ]:
# load data and set numpy array

import numpy as np

# load data
loaded_data = np.load('js115_data.npy')
print(f"-- load data shape : {loaded_data.shape}")

# set numpy array
S = loaded_data[0]
E = loaded_data[1]
I = loaded_data[2]
R = loaded_data[3]
NewCase = loaded_data[5]

In [ ]:
# plot S-E-I-R NewCase*50

import matplotlib.pyplot as plt

plt.plot(S, 'b-')
plt.plot(E, 'm-')
plt.plot(I, 'r-')
plt.plot(R, 'g-')
plt.plot(NewCase * 50, 'c-')
plt.title('S-E-I-R Monte-Carlo Simulation Model')
plt.xlabel('time( day )')
plt.ylabel('population')
plt.grid()
plt.show()

In [ ]:
# prompt: plot S, E, I, R and NewCase*50 with plotly

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=S, mode='lines', name='S'))
fig.add_trace(go.Scatter(y=E, mode='lines', name='E'))
fig.add_trace(go.Scatter(y=I, mode='lines', name='I'))
fig.add_trace(go.Scatter(y=R, mode='lines', name='R'))
fig.add_trace(go.Scatter(y=NewCase * 50, mode='lines', name='NewCase*50'))

fig.update_layout(title='S-E-I-R Monte-Carlo Simulation Model',
                  xaxis_title='time(day)',
                  yaxis_title='population')
fig.show()

In [ ]:
# prompt: plot smoothed NewCase(moving average 7) and detect max and date

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'NewCase' is already defined from the previous code
# Create a pandas Series from NewCase
new_cases_series = pd.Series(NewCase)

# Calculate the 7-day moving average
smoothed_new_cases = new_cases_series.rolling(window=7).mean()

# Find the index of the maximum smoothed new case
max_index = smoothed_new_cases.idxmax()

# Get the maximum smoothed new case value
max_new_case = smoothed_new_cases.max()


# Plotting
plt.figure(figsize=(10, 6))
plt.plot(smoothed_new_cases, label='Smoothed New Cases (7-day MA)')
plt.scatter(max_index, max_new_case, color='red', label=f'Max: {max_new_case:.2f} at day {max_index}')

plt.title('Smoothed New Cases with Maximum Detection')
plt.xlabel('Day')
plt.ylabel('Number of New Cases')
plt.legend()
plt.grid(True)
plt.show()

print(f"Maximum smoothed new cases: {max_new_case:.2f}")
print(f"Day of maximum smoothed new cases: {max_index}")

In [ ]:
# prompt: data analysis S E I R corr, scatter with pandas

import pandas as pd

# Create a Pandas DataFrame from the loaded data
df = pd.DataFrame({'S': S, 'E': E, 'I': I, 'R': R})

# Calculate the correlation matrix
corr_matrix = df.corr()
print("Correlation Matrix:")
print(corr_matrix)

# Create a scatter plot matrix
pd.plotting.scatter_matrix(df, alpha=0.2)
plt.show()


In [ ]:
# find peak of I
# colab AI wrote:
# prompt: get peak top of I and time (day)

# Find the index of the peak value in the I array
peak_index = np.argmax(I)

# Get the peak value and the corresponding time (day)
peak_value = I[peak_index]
peak_time = peak_index

print("Peak of I:", peak_value)
print("Time (day) at peak:", peak_time)


In [ ]:
# prompt: scatter plot I and new cases

# Scatter plot of I and New Cases
plt.scatter(I, NewCase, alpha=0.5)
plt.xlabel('Infected Population (I)')
plt.ylabel('New Cases')
plt.title('Scatter Plot of Infected Population and New Cases')
plt.show()


In [ ]:
# prompt: get corr coef

# Calculate the correlation coefficient between I and NewCase
corr_coef = np.corrcoef(I, NewCase)[0, 1]

print("Correlation Coefficient between I and NewCase:", corr_coef)


In [ ]:
# prompt: get coef in equation I = coef * NewCase

# Calculate the slope of the linear regression line
slope, _ = np.polyfit(NewCase, I, 1)

# The coefficient in the equation I = coef * NewCase is the slope
coef = slope

print("Coefficient in the equation I = coef * NewCase:", coef)
